In [1]:
# 📌 **1. Import Library**
import requests
import json
import os
import base64
import pandas as pd
import tensorflow as tf

# 📌 **2. Baca Data dari CSV**
csv_path = "data/anxiety_attack_dataset.csv"
data = pd.read_csv(csv_path)

# 📌 **3. Ambil 10 Data Secara Acak untuk Uji**
sample_data = data.sample(n=10, random_state=42)  # Pilih 10 baris acak untuk diuji

In [2]:
# 📌 **4. Transformasi Sesuai dengan TFX Pipeline**

# Daftar fitur pada dataset yang perlu di-encode
FEATURES = [
    "Age",
    "Sleep Hours",
    "Physical Activity (hrs/week)",
    "Caffeine Intake (mg/day)",
    "Alcohol Consumption (drinks/week)",
    "Stress Level (1-10)",
    "Heart Rate (bpm during attack)",
    "Breathing Rate (breaths/min)",
    "Sweating Level (1-5)",
    "Therapy Sessions (per month)",
    "Diet Quality (1-10)",
    "Gender",
    "Occupation",
    "Smoking",
    "Family History of Anxiety",
    "Dizziness",
    "Medication",
    "Recent Major Life Event"
]

# Label key
LABEL_KEY = "Anxiety Category Encoded"

In [8]:
inputs_df = sample_data[FEATURES + [LABEL_KEY]]

In [13]:
# 📌 **5. Fungsi untuk Menyiapkan JSON untuk API**
def prepare_json(inputs: dict):
    feature_mapping = {
        key: tf.train.Feature(float_list=tf.train.FloatList(value=[float(inputs[key])])) 
        for key in inputs.keys()
    }

    example = tf.train.Example(features=tf.train.Features(feature=feature_mapping)).SerializeToString()
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }

    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result],
    })

In [14]:
# 📌 **6. Fungsi untuk Mengirim Request ke API dengan Pengecekan yang Lebih Detail**
def predict(inputs):
    json_data = prepare_json(inputs)

    # URL endpoint untuk model yang sudah dideploy
    endpoint = "https://ml-pipeline-anxiety-attack-production.up.railway.app/v1/models/anxiety-model:predict"

    response = requests.post(endpoint, data=json_data)
    
    # Pengecekan apakah request berhasil (status 200) dan respons valid
    if response.status_code == 200:
        try:
            predictions = response.json().get("predictions")
            if predictions:
                return int(predictions[0])  # Hasil prediksi berupa angka 0, 1, 2, atau 3
            else:
                return "Error: No predictions found in response."
        except ValueError:
            return "Error: Invalid JSON response."
    else:
        # Menampilkan lebih banyak informasi dari respons jika status bukan 200
        try:
            error_info = response.json()
            return f"Error: API request failed with status code {response.status_code}. Message: {error_info.get('error', 'No error message provided')}"
        except ValueError:
            return f"Error: API request failed with status code {response.status_code}. No JSON response."
    
# 📌 **7. Melakukan Prediksi untuk 10 Data Acak**
for idx, row in transformed_data.iterrows():
    inputs = row.to_dict()
    print(f"Prediksi untuk baris {idx}: {predict(inputs)}")

Prediksi untuk baris 0: Error: API request failed with status code 400. Message: Name: <unknown>, Feature: Age (data type: int64) is required but could not be found.
	 [[{{function_node __inference_serve_tf_examples_fn_341176}}{{node ParseExample/ParseExampleV2}}]]
Prediksi untuk baris 1: Error: API request failed with status code 400. Message: Name: <unknown>, Feature: Age (data type: int64) is required but could not be found.
	 [[{{function_node __inference_serve_tf_examples_fn_341176}}{{node ParseExample/ParseExampleV2}}]]
Prediksi untuk baris 2: Error: API request failed with status code 400. Message: Name: <unknown>, Feature: Age (data type: int64) is required but could not be found.
	 [[{{function_node __inference_serve_tf_examples_fn_341176}}{{node ParseExample/ParseExampleV2}}]]
Prediksi untuk baris 3: Error: API request failed with status code 400. Message: Name: <unknown>, Feature: Age (data type: int64) is required but could not be found.
	 [[{{function_node __inference_serv